In [2]:
import pandas as pd

In [3]:
# Importacion de Plotly como libreria de DataViz
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [4]:
import os

# Obtiene la ubicación completa del archivo del notebook
notebook_path = os.path.realpath('__file__')

# Obtiene la ruta absoluta de la carpeta "Coderhouser-DS-TP0002" sin importar donde te encuentres
path_base = os.path.abspath(os.path.join(notebook_path, "..", "..", "..", "Coderhouser-DS-TP0002"))

print("Path base':", path_base)

Path base': C:\develoment\Coderhouser-DS-TP0002


#### Leyendo datos:

In [14]:
# Generando el Dataframe apartir de los dataset RedTeam y BlueTeam
blueTeam = pd.read_csv( path_base + r"\\data\procesed\blueTeam.csv")
blueTeam["Team"] = "Blue"
redTeam = pd.read_csv( path_base + r"\\data\procesed\redTeam.csv")
redTeam["Team"] = "Red"
data = pd.concat([blueTeam,redTeam])

In [30]:
# Reemplazar los valores de la columna "Wins"
data['WinTags'] = data['Wins'].replace({0: 'Loss', 1: 'Win'})

# Crear un mapeo de colores
color_map = {'Win': 'yellow', 'Loss': 'gray'}

In [15]:
# Detalle de tabla
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19758 entries, 0 to 9878
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                19758 non-null  int64  
 1   Wins                      19758 non-null  int64  
 2   WardsPlaced               19758 non-null  int64  
 3   WardsDestroyed            19758 non-null  int64  
 4   FirstBlood                19758 non-null  int64  
 5   Kills                     19758 non-null  int64  
 6   Deaths                    19758 non-null  int64  
 7   Assists                   19758 non-null  int64  
 8   EliteMonsters             19758 non-null  int64  
 9   Dragons                   19758 non-null  int64  
 10  Heralds                   19758 non-null  int64  
 11  TowersDestroyed           19758 non-null  int64  
 12  TotalGold                 19758 non-null  int64  
 13  AvgLevel                  19758 non-null  float64
 14  TotalEx

### Hipotesis
Estas hipótesis proporcionan una base sólida para realizar pruebas y análisis más detallados sobre el dataset de League of Legends. Al evaluar cada una de estas hipótesis y validarlas con datos, el equipo podrá obtener una mejor comprensión de los factores que realmente influyen en el rendimiento y la probabilidad de victoria en el juego.

- **Hipótesis 1**: Control temprano del mapa y victoria Equipos que logran un mayor control del mapa mediante la colocación de más "Warding totems" en áreas estratégicas durante los primeros 10 minutos tendrán una mayor probabilidad de ganar la partida.

- **Hipótesis 2**: Relación entre asesinatos y tasa de victoria El número de asesinatos conseguidos por un equipo en los primeros 10 minutos se correlacionará positivamente con su tasa de victoria. Cuanto mayor sea la cantidad de asesinatos, más alta será la probabilidad de ganar.

- **Hipótesis 3**: Impacto de objetivos neutrales en la victoria Equipos que aseguran más dragones y Herald durante los primeros 10 minutos tendrán una mayor probabilidad de ganar la partida, ya que estos objetivos proporcionan beneficios significativos para todo el equipo.

- **Hipótesis 4**: Importancia de la ventaja económica y de experiencia Equipos que logran una ventaja significativa en oro y experiencia sobre el equipo contrario en los primeros 10 minutos tendrán mayores posibilidades de obtener la victoria.


- **Hipótesis 5**: Contribución individual y tasa de victoria Jugadores que contribuyen con un mayor número de asistencias tendrán un impacto positivo en la tasa de victoria de su equipo en las partidas clasificatorias Diamante.

- **Hipótesis 6**: Roles clave para el éxito Identificar roles específicos, como un jungla efectivo o un carry con muchas asistencias, será crucial para el éxito del equipo y aumentará sus posibilidades de ganar.

- **Hipótesis 7**: Nivel promedio y probabilidad de victoria Un equipo con un mayor nivel promedio de sus jugadores durante los primeros 10 minutos tendrá una mayor probabilidad de ganar la partida.

- **Hipótesis 8**: Adaptabilidad y éxito Equipos que pueden ajustar sus tácticas y estrategias según la situación en los primeros 10 minutos tendrán una mayor probabilidad de lograr la victoria en partidas clasificatorias Diamante.

In [41]:
# Crear el gráfico de dispersión
fig = px.scatter(data, x='GoldDiff', y='ExperienceDiff', color='WinTags', color_discrete_map=color_map,
                 labels={'GoldDiff': 'Diferencia de Oro', 'ExperienceDiff': 'Diferencia de Experiencia', 'WinTags': 'Resultado de la Partida'},
                 title="Relación entre Diferencia de Oro, Diferencia de Experiencia y Resultado de la Partida")

# Personalizar el gráfico
fig.update_layout(
    xaxis_title="Diferencia de Oro",
    yaxis_title="Diferencia de Experiencia"
)

# Cambiar las etiquetas de la leyenda
fig.update_traces(marker=dict(size=6), selector=dict(name=0), legendgroup="Loss", showlegend=True)
fig.update_traces(marker=dict(size=6), selector=dict(name=1), legendgroup="Win", showlegend=True)

# Mostrar el gráfico
fig.show()